In [48]:
import pandas as pd
import numpy as np
import statsmodels

In [49]:
cases = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_fact.csv')
cases_by_account = cases.groupby('ACCOUNTKEY')['CASE_NUMBER'].agg(['count'])
print(cases_by_account.head)

<bound method NDFrame.head of                                   count
ACCOUNTKEY                             
000018da887ac952b77d51c1e6826a4a     13
000682dbae7a366c6acece0989c5251b     11
000e29f3849ed00bf0d7a819016c24ee      3
0013a2c7fb8854e45b6656d236b62a5a     17
0017433f167d6004b0356f8f21d0c08a      1
...                                 ...
fffbea9bb73eae66096240121bca0a55     47
fffc42e981785ef6df8b21d30e5389f5      1
fffcc63901bbf695985ef18b0a0f4b40      4
ffff15c523999dce650e7fa84b5eba62      2
ffff4cc2e89bbf08e5f5a0aa953a7613      2

[32564 rows x 1 columns]>


In [50]:
case_oppertunity = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/case_oppertunities.csv')
case_text = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/support_cases_text_fact.csv')
print(case_oppertunity.shape)
print(case_oppertunity['CASE_NUMBER'].nunique())
case_oppertunity['completetion_time'] = pd.to_datetime(case_oppertunity['CLOSED_DATE']) - pd.to_datetime(case_oppertunity['CREATED_DATE'])

(183669, 35)
183669


In [51]:
case_oppertunity = case_oppertunity.merge(cases_by_account,
                                          on = 'ACCOUNTKEY',
                                          how='left')
print(case_oppertunity.columns)

Index(['Unnamed: 0', 'STATUS', 'PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT',
       'CATEGORY', 'SUBCATEGORY', 'CREATED_DATE', 'CLOSED_DATE',
       'CASE_CLOSED_REASON', 'RECORDTYPE', 'CASE_NUMBER', 'ACCOUNTKEY',
       'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL', 'EMPLOYEECOUNT', 'THEATER',
       'SUBREGION', 'SEGMENT', 'SUBSEGMENT', 'Tenure', 'Zero.Revenue.Flag',
       'Einstein Analytics', 'Role Based - Online', 'Server', 'Saas',
       'Role Based - On Prem', 'Services', 'Desktop', 'Role Based - NonProd',
       'Other', 'CLOSEDATE', 'ISWON', 'ISPARTNERRELATED', 'completetion_time',
       'count'],
      dtype='object')


In [52]:
case_oppertunity['completetion_time'] = case_oppertunity['completetion_time'].astype('int')

/var/folders/t_/rcfcs8g56jn7trwnsvmdyh_r0000gn/T/ipykernel_17523/2754767065.py:1: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  case_oppertunity['completetion_time'] = case_oppertunity['completetion_time'].astype('int')


In [54]:
case_categories = case_oppertunity[['STATUS','PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT',
                                    'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL','THEATER',
                                    'SUBREGION', 'SEGMENT', 'SUBSEGMENT']]
case_descrete = case_oppertunity[['Tenure', 'completetion_time', 'count', 'EMPLOYEECOUNT']]
case_one_hot_encode = pd.get_dummies(case_categories)
case_features = pd.concat([case_one_hot_encode, 
                           case_descrete],
                          axis=1)
print(case_features.shape)
labels = case_oppertunity['ISWON']
case_features.drop(case_features.columns[169], axis=1, inplace=True)
case_features['completetion_time'] = case_features['completetion_time'].astype('int')

(183669, 170)


In [ ]:
print(case_features.describe())

In [65]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(case_descrete)
scaled_case = pd.DataFrame(data_scaled, 
                         columns = case_descrete.columns)
case_causal_features = pd.concat([scaled_case, case_one_hot_encode], axis=1)
log_model = sm.OLS(labels, case_causal_features).fit()
print(log_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  ISWON   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     155.4
Date:                Thu, 12 May 2022   Prob (F-statistic):               0.00
Time:                        18:32:25   Log-Likelihood:            -1.0578e+05
No. Observations:              183669   AIC:                         2.119e+05
Df Residuals:                  183518   BIC:                         2.134e+05
Df Model:                         150                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------